In [1]:
!nvidia-smi

Mon Dec 27 05:24:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:01:00.0 Off |                    0 |
| N/A   51C    P0    68W / 275W |  13297MiB / 40536MiB |     40%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  Off  | 00000000:47:00.0 Off |                    0 |
| N/A   

In [1]:
!pip install torchio

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 164 kB 700 kB/s eta 0:00:01
     |████████████████████████████████| 48.4 MB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 98 kB 9.9 MB/s eta 0:00:011
     |████████████████████████████████| 3.3 MB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 10.5 MB/s ta 0:00:011


In [3]:
!pip install monai

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 709 kB 221 kB/s eta 0:00:01


In [1]:
import random
import os
import os.path

import numpy as np
import math
import tqdm
import random
import copy
import torchio

import torch as t
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F 

import cv2
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter



In [2]:
t.backends.cudnn.enabled = True
#os.environ["CUDA_VISIBLE_DEVICES"]="1"
#device = t.device('cuda' if t.cuda.is_available() else 'cpu')

#device = t.device("cuda:2")

device = t.device("cuda:2")
device1 = t.device("cuda:2")
device2 = t.device("cuda:1")
print (device)
#device = "cuda:0"
#device1 = "cuda:0"
#device2 = "cuda:1"

epoch150_path_name = "MICCCAI_tcia_150.pth"
epoch150_50_path_name = "MICCAI_tcia_150+50.pth"


file_name = "MICCAI_dice_log_tcia.txt"
file_name_1 = "MICCAI_loss_log_tcia.txt"
file_name_2 = "MICCAI_totalDice_log_tcia.txt"

y_pred_path = "OutputsTcia/MICCAI_pred_tcia"
y_true_path = "OutputsTcia/MICCAI_true_tcia"


cuda:2


In [3]:
# # Encoder - Decoder Network
class ChannelPool(nn.Module):
    def forward(self,x):
        return t.cat((t.max(x,1)[0].unsqueeze(1), t.mean(x,1).unsqueeze(1)),dim=1)

class SpatialGate(nn.Module):

    def __init__(self):
        super().__init__()
        self.compress = ChannelPool()
        self.conv = nn.Sequential(
            nn.Conv3d(2,1,kernel_size=7,stride=1,padding=3, bias=False),
            nn.BatchNorm3d(1,eps=1e-5,momentum=0.01,affine=True),
            nn.Sigmoid()
        )
    def forward(self,x):
        xcompress=self.compress(x)
        spatialAttention=self.conv(xcompress)
        return x*spatialAttention
    
class Flatten(nn.Module):
    def forward(self,x):
        return x.view(x.size(0),-1)

class ChannelGate(nn.Module):
  
    def __init__(self,channels,reductionRatio=16,poolTypes=['avg','max']):
        super().__init__()
        self.channels = channels
        self.mlp = nn.Sequential(
            Flatten(),
            nn.Linear(channels,channels//reductionRatio),
            nn.ReLU(),
            nn.Linear(channels//reductionRatio,channels)
        )
        self.poolTypes = poolTypes
  
    def forward(self,x):
        attentionSum = None
        for poolType in self.poolTypes:
            if poolType=='avg':
                avgPool = F.avg_pool3d(x,(x.size(2),x.size(3),x.size(4)),stride=(x.size(2),x.size(3),x.size(4)))
                channelAttention = self.mlp(avgPool)
            if poolType=='max':
                maxPool = F.max_pool3d(x,(x.size(2),x.size(3),x.size(4)),stride=(x.size(2),x.size(3),x.size(4)))
                channelAttention = self.mlp(maxPool)
        if attentionSum is None:
              attentionSum = channelAttention
        else:
              attentionSum=attentionSum+channelAttention
    
        scale=t.sigmoid(attentionSum).unsqueeze(2).unsqueeze(3).unsqueeze(4).expand_as(x)
        return x*scale


class CBAM(nn.Module):
  
    def __init__(self,channels,reductionRatio=16,poolTypes=['avg','max']):
        super().__init__()
        self.channelGate = ChannelGate(channels,reductionRatio,poolTypes)
        self.spatialGate = SpatialGate()

    def forward(self,x):
        x=self.spatialGate(x)
        x=self.channelGate(x)
        return x 
    
class CBAMResnetBlock(nn.Module):

    def __init__(self, in_channels, out_channels,kernelSize=3):
        super().__init__()
        self.resnetblock1 = nn.Sequential(
            nn.Conv3d(in_channels,out_channels,kernel_size=kernelSize,padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=False)
            )
        self.resnetblock2 = nn.Sequential(
            nn.Conv3d(out_channels,out_channels,kernel_size=kernelSize,padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=False),
            nn.Conv3d(out_channels,out_channels,kernel_size=kernelSize,padding=1),
            nn.BatchNorm3d(out_channels)
            )
        self.CBAM = CBAM(out_channels,16,['avg','max'])
        self.reluUnit = nn.Sequential(
            nn.ReLU(inplace=False)
            )

    def forward(self,x):
        x=self.resnetblock1(x)
        x1=x
        x=self.resnetblock2(x)
        x=self.CBAM(x)
        x=x+x1
        x=self.reluUnit(x)
        return x
    
class GSEncoder(nn.Module):
    
    def __init__(self):
        
        super().__init__()
        self.downConv  = nn.Sequential(
            nn.Conv3d(1,32,kernel_size=3,stride=2,padding=1,bias=False),
            nn.BatchNorm3d(32),
            nn.LeakyReLU(inplace=False),
        )

        self.layer21 = nn.Sequential(
            CBAMResnetBlock(32,40),
            CBAMResnetBlock(40,40)
        )

        self.layer22 = nn.Sequential(
            CBAMResnetBlock(40,48),
            CBAMResnetBlock(48,48)
        )

        self.layer23 = CBAMResnetBlock(48,56)

    def forward(self,x):

        x0=self.downConv(x)
        x1=self.layer21(x0)
        x2=self.layer22(x1)
        x3=self.layer23(x2)
        #print(x3.shape)
        return x,x0,x1,x2,x3


class GSDecoder_part1(nn.Module):
    
    def __init__(self):
        super().__init__()
    
        self.layer23 = nn.Sequential(
            CBAMResnetBlock(56,56),
            CBAMResnetBlock(56,48)
        )
        self.layer22 = nn.Sequential(
            CBAMResnetBlock(96,48),
            CBAMResnetBlock(48,48),
            CBAMResnetBlock(48,40)
        )
        self.layer21 = nn.Sequential(
            CBAMResnetBlock(80,40),
            CBAMResnetBlock(40,40),
            CBAMResnetBlock(40,32)
        )
        self.layer20 = nn.Sequential(
            CBAMResnetBlock(64,32),
            CBAMResnetBlock(32,32)
        )
        self.transposeConv = nn.ConvTranspose3d(32,16,kernel_size=2,stride=2)
        self.layer10 = nn.Conv3d(17,16,kernel_size=3,padding=1)

    def forward(self,x,x0,x1,x2,x3):
        
        y=self.layer23(x3)
        y=t.cat([y,x2],dim=1)
        y=self.layer22(y)
        y=t.cat([y,x1],dim=1)
        y=self.layer21(y)
        y=t.cat([y,x0],dim=1)
        y=self.layer20(y)
        y=self.transposeConv(y)
        y=t.cat([y,x],dim=1)
        y=self.layer10(y)
        #print(y.shape)
        return y


class GSNetCore(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.encoder = GSEncoder() .to(device2)
        self.decoder_part1 = GSDecoder_part1().to(device2)
  
    def forward(self,x):
        x,x0,x1,x2,x3=self.encoder(x)
        y=self.decoder_part1(x,x0,x1,x2,x3)
        return y


class GSNetModel(nn.Module):
  
    def __init__(self):
        super().__init__()
        self.core = GSNetCore()
        self.finalConv = nn.Sequential(
            nn.Conv3d(16,8,1),
            nn.Softmax(dim=1),
        ).cuda()
  
    def forward(self,x):
        y=self.core(x)
        y=self.finalConv(y)
        #print(y.shape)
        return y
    

In [4]:
class HaN_Dataset(Dataset):
    
    def __init__(self, root_dir=None, transform=False, alpha=1000, sigma=30, alpha_affine=0.04):
        super().__init__()
        self.path = root_dir
        self.datas = t.load(self.path)
        
        self.transform = transform
        self.alpha = alpha
        self.sigma = sigma
        self.alpha_affine = alpha_affine
    
    def __getitem__(self, index):
        data = self.datas[index]
        img = data['img'].numpy().astype(np.float32)
        
        if not self.transform:
            masklst = []
            for mask in data['mask']:
                if mask is None:
                    mask = np.zeros((1,img.shape[0],img.shape[1],img.shape[2])).astype(np.uint8)
                masklst.append(mask.astype(np.uint8).reshape((1,img.shape[0],img.shape[1],img.shape[2]))) 
            mask0 = np.zeros_like(masklst[0]).astype(np.uint8)
            for mask in masklst:
                mask0 = np.logical_or(mask0, mask).astype(np.uint8)
            mask0 = 1 - mask0
            return t.from_numpy(img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))), t.from_numpy(np.concatenate([mask0]+masklst[0:7], axis=0)), True
        
       
        im_merge = np.concatenate([img[...,None]]+[mask.astype(np.float32)[...,None] for mask in data['mask']], axis=3)
        # Apply transformation on image
        im_merge_t, new_img = self.elastic_transform3Dv2(im_merge,self.alpha,self.sigma,min(im_merge.shape[1:-1])*self.alpha_affine)
        # Split image and mask ::2, ::2, ::2
        im_t = im_merge_t[...,0]
        im_mask_t = im_merge_t[..., 1:].astype(np.uint8).transpose(3, 0, 1, 2)
        mask0 = np.zeros_like(im_mask_t[0, :, :, :]).reshape((1,)+im_mask_t.shape[1:]).astype(np.uint8)
        im_mask_t_lst = []
        flagvect = np.ones((8,), np.float32)
        retflag = True
        for i in range(7):
            im_mask_t_lst.append(im_mask_t[i,:,:,:].reshape((1,)+im_mask_t.shape[1:]))
            if im_mask_t[i,:,:,:].max() != 1: 
                retflag = False
                flagvect[i+1] = 0
            mask0 = np.logical_or(mask0, im_mask_t[i,:,:,:]).astype(np.uint8)
        if not retflag: flagvect[0] = 0
        mask0 = 1 - mask0
        return t.from_numpy(im_t.reshape((1,)+im_t.shape[:3])), t.from_numpy(np.concatenate([mask0]+im_mask_t_lst, axis=0)), flagvect
        
    def __len__(self):
        return len(self.datas)
    
    def elastic_transform3Dv2(self, image, alpha, sigma, alpha_affine, random_state=None):
        """Elastic deformation of images as described in [Simard2003]_ (with modifications).
        .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
             Convolutional Neural Networks applied to Visual Document Analysis", in
             Proc. of the International Conference on Document Analysis and
             Recognition, 2003.
         Based on https://gist.github.com/erniejunior/601cdf56d2b424757de5
         From https://www.kaggle.com/bguberfain/elastic-transform-for-data-augmentation
        """
        # affine and deformation must be slice by slice and fixed for slices
        if random_state is None:
            random_state = np.random.RandomState(None)
        shape = image.shape # image is contatenated, the first channel [:,:,:,0] is the image, the second channel 
        # [:,:,:,1] is the mask. The two channel are under the same tranformation.
        shape_size = shape[:-1] # z y x
        # Random affine
        shape_size_aff = shape[1:-1] # y x
        center_square = np.float32(shape_size_aff) // 2
        square_size = min(shape_size_aff) // 3
        pts1 = np.float32([center_square + square_size, [center_square[0]+square_size, center_square[1]-square_size], center_square - square_size])
        pts2 = pts1 + random_state.uniform(-alpha_affine, alpha_affine, size=pts1.shape).astype(np.float32)
        M = cv2.getAffineTransform(pts1, pts2)
        new_img = np.zeros_like(image)
        for i in range(shape[0]):
            new_img[i,:,:,0] = cv2.warpAffine(image[i,:,:,0], M, shape_size_aff[::-1], borderMode=cv2.BORDER_CONSTANT, borderValue=0.)
            for j in range(1, 8):
                new_img[i,:,:,j] = cv2.warpAffine(image[i,:,:,j], M, shape_size_aff[::-1], flags=cv2.INTER_NEAREST, borderMode=cv2.BORDER_TRANSPARENT, borderValue=0)
        dx = gaussian_filter((random_state.rand(*shape[1:-1]) * 2 - 1), sigma) * alpha
        dy = gaussian_filter((random_state.rand(*shape[1:-1]) * 2 - 1), sigma) * alpha
        x, y = np.meshgrid(np.arange(shape_size_aff[1]), np.arange(shape_size_aff[0]))
        indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1))
        new_img2 = np.zeros_like(image)
        for i in range(shape[0]):
            new_img2[i,:,:,0] = map_coordinates(new_img[i,:,:,0], indices, order=1, mode='constant').reshape(shape[1:-1])
            for j in range(1, 8):
                new_img2[i,:,:,j] = map_coordinates(new_img[i,:,:,j], indices, order=0, mode='constant').reshape(shape[1:-1])
        return np.array(new_img2), new_img
      
# %%


In [5]:
#loading MiCCAi data
traindataset = HaN_Dataset("Data/train_data38.pth", transform=True)
traindataloader = DataLoader(traindataset, batch_size=1, shuffle=True)
testdataset = HaN_Dataset("Data/test_data.pth", transform=False)
testdataloader = DataLoader(testdataset, batch_size=1)

print(len(testdataloader), len(traindataloader))

10 38


In [6]:
def crossentropy(y_pred, y_true, flagvec, device):
    retv = - t.sum(t.sum(t.sum(t.sum(t.log(t.clamp(y_pred,1e-6,1))*y_true.type(t.cuda.FloatTensor),4),3),2),0) * flagvec.to(device)
    return t.sum(retv / (t.sum(t.sum(t.sum(t.sum(y_true.type(t.cuda.FloatTensor),4),3),2),0) + 1e-6)) / y_true.size()[1]


# %%


def tversky_loss_wmask(y_pred, y_true, flagvec, device):
    alpha = 0.5
    beta  = 0.5
    ones = t.ones_like(y_pred) 

    p0 = y_pred      # proba that voxels are class i
    p1 = ones-y_pred # proba that voxels are not class i
    g0 = y_true.type(t.cuda.FloatTensor)
    g1 = ones-g0
    num = t.sum(t.sum(t.sum(t.sum(p0*g0, 4),3),2),0) #(0,2,3,4)) #K.sum(p0*g0, (0,1,2,3))
    den = num + alpha*t.sum(t.sum(t.sum(t.sum(p0*g1,4),3),2),0) + beta*t.sum(t.sum(t.sum(t.sum(p1*g0,4),3),2),0) #(0,2,3,4))

    T = t.sum((num* flagvec.to(device))/(den+1e-5))
    return t.sum(flagvec.to(device))-T

# %%

def focal(y_pred, y_true, flagvec, device):
    retv = - t.mean(t.mean(t.mean(t.mean(t.log(t.clamp(y_pred,1e-6,1))*y_true.type(t.cuda.FloatTensor)*t.pow(1-y_pred,2),4),3),2),0) * flagvec.to(device)
    return t.sum(retv)


In [7]:
def caldice(y_pred, y_true):

    y_pred = y_pred.data.cpu().numpy().transpose(1,0,2,3,4) # inference should be arg max
    y_pred = np.argmax(y_pred, axis=0).squeeze() # z y x
    y_true = y_true.data.cpu().numpy().transpose(1,0,2,3,4).squeeze() # .cpu()
    avgdice = []
    y_pred_1 = y_pred==1
    y_true_1 = y_true[1,:,:,:]
    if y_pred_1.sum() + y_true_1.sum() == 0: avgdice.append(-1)
    else: avgdice.append(2.*(np.logical_and(y_pred_1, y_true_1).sum()) / (1.0*(y_pred_1.sum() + y_true_1.sum())))
    
    y_pred_1 = y_pred==2
    y_true_1 = y_true[2,:,:,:]
    if y_pred_1.sum() + y_true_1.sum() == 0: avgdice.append(-1)
    else: avgdice.append(2.*(np.logical_and(y_pred_1, y_true_1).sum()) / (1.0*(y_pred_1.sum() + y_true_1.sum())))
    
    y_pred_1 = y_pred==3
    y_true_1 = y_true[3,:,:,:]
    if y_pred_1.sum() + y_true_1.sum() == 0: avgdice.append(-1)
    else: avgdice.append(2.*(np.logical_and(y_pred_1, y_true_1).sum()) / (1.0*(y_pred_1.sum() + y_true_1.sum())))
    
    y_pred_1 = y_pred==4
    y_true_1 = y_true[4,:,:,:]
    if y_pred_1.sum() + y_true_1.sum() == 0: avgdice.append(-1)
    else: avgdice.append(2.*(np.logical_and(y_pred_1, y_true_1).sum()) / (1.0*(y_pred_1.sum() + y_true_1.sum())))
    
    y_pred_1 = y_pred==5
    y_true_1 = y_true[5,:,:,:]
    if y_pred_1.sum() + y_true_1.sum() == 0: avgdice.append(-1)
    else: avgdice.append(2.*(np.logical_and(y_pred_1, y_true_1).sum()) / (1.0*(y_pred_1.sum() + y_true_1.sum())))
    
    y_pred_1 = y_pred==6
    y_true_1 = y_true[6,:,:,:]
    if y_pred_1.sum() + y_true_1.sum() == 0: avgdice.append(-1)
    else: avgdice.append(2.*(np.logical_and(y_pred_1, y_true_1).sum()) / (1.0*(y_pred_1.sum() + y_true_1.sum())))
    
    y_pred_1 = y_pred==7
    y_true_1 = y_true[7,:,:,:]
    if y_pred_1.sum() + y_true_1.sum() == 0: avgdice.append(-1)
    else: avgdice.append(2.*(np.logical_and(y_pred_1, y_true_1).sum()) / (1.0*(y_pred_1.sum() + y_true_1.sum())))

    for dice in avgdice: 
        if dice != -1:
            assert 0 <= dice <= 1
    return avgdice

def get_yPredDash(y_pred):
    
    y_pred = y_pred.data.cpu().numpy() # inference should be arg max
    y_pred_1 = np.zeros_like(y_pred).astype(np.uint8)
    y_pred = np.argmax(y_pred, axis=1).squeeze() # z y x
    
    for i in range(8):
        y_pred_1[:,i,:,:,:] = y_pred==i
    
    return y_pred_1


In [8]:
#loading the network with trained weights
import time
SSABAM_Model = GSNetModel().cuda()
SSABAM_Model.to(device)

loaded_model = t.load("Weights/swap10_tcia_120+80.pth", map_location='cuda:0')
SSABAM_Model.load_state_dict(loaded_model["model_state_dict"])
print("Weights loaded")

Weights loaded


In [9]:
#training the network with MICCAI data

lossweight = np.array([2.22, 1.06, 1.02, 1.74, 1.93, 1.93, 1.13, 1.15], np.float32)
ceweight = 0.05
focweight = 0.5
savename = 'model/MICCAI_Train_New_'

optimizer = t.optim.RMSprop(SSABAM_Model.parameters(),lr = 2e-3)

# %%
epoch1=150
epoch2=50
#for epoch in range(anatomy_epoch1):
maxloss = [0 for _ in range(7)]
for epoch in range(epoch1):
    i=0
    j=0
    train_loss = 0
    print("epoch ", epoch+1)
    for x_train, y_train, flagvec in tqdm.tqdm(traindataloader):
        #Anatomy_Model.train()
        i+=1

        #print(f'epoch {epoch+1} batch {i} tensor_shape {x_train.shape}')
        
        """
        if x_train.shape[2] > 150:
            x_train = x_train[:,:,37:37+64,...]
            y_train = y_train[:,:,37:37+64,...]
        """ 
        if x_train.shape[-2] > 200:
            x_train = x_train[...,10:,:]
            y_train = y_train[...,10:,:]
        
        x_train = x_train.to(device1) #(device1)
        out = SSABAM_Model(x_train)
        
        out = out.to(device2) #(device2)
        y_train = y_train.to(device2) #(device2)
        loss = tversky_loss_wmask(out, y_train, flagvec*t.from_numpy(lossweight),device2) # device2)
        celoss = crossentropy(out, y_train, flagvec*t.from_numpy(lossweight), device2) # device2)
        floss = focal(out, y_train, flagvec*t.from_numpy(lossweight), device2 )  # device2)
        optimizer.zero_grad()
        (loss + (ceweight*celoss) + (focweight*floss)).backward()
        optimizer.step()
        train_loss += loss.item() + (ceweight*celoss).item() + (focweight*floss).item()

        y_train.detach_()
        out.detach_()
        del loss, x_train, y_train, out, floss, celoss
        
    print("train loss: ",train_loss/(len(traindataloader)-j),'\n')
    
    testtq = tqdm.tqdm(testdataloader)#, desc='loss', leave=True)
    test_loss = 0
    testloss = [0 for _ in range(7)]
    for x_test, y_test, flagvecTest in testtq:
        with t.no_grad():
            #Anatomy_Model.eval()
            x_test = x_test.to(device1) #(device1)
            y_test = y_test.to(device2) #(device2)
            out = SSABAM_Model(x_test)
            out = out.to(device2) #(device2)
            #loss = tversky_loss_wmask(out, y_test, flagvecTest*t.from_numpy(lossweight),device2) # device2)
            #celoss = crossentropy(out, y_test, flagvecTest*t.from_numpy(lossweight), device2) # device2)
            #floss = focal(out, y_test, flagvecTest*t.from_numpy(lossweight),device2) #device2)
            #test_loss += loss.item() + (ceweight*celoss).item() + (focweight*floss).item()
            dice = caldice(out, y_test)
            testtq.set_description("test loss %f" % (sum(dice)/7))
            testtq.refresh() # to show immediately the update
            testloss = [l+tl if l != -1 else tl for l,tl in zip(dice, testloss)]  #testloss = [l+tl for l,tl in zip(loss, testloss)]
            del out, x_test, y_test
            
    print(f"test loss: {test_loss/len(testdataloader)}")
    testloss = [l / len(testtq) for l in testloss]
    for cls in range(7):
        if maxloss[cls] < testloss[cls]:
            maxloss[cls] = testloss[cls]
            state = {"epoch": epoch, "weight": SSABAM_Model.state_dict()}
            t.save(state, savename+str(cls+1))
#             model.load_state_dict(t.load(savename)["weight"])
#             t.save(model, savename+str(cls+1))
    print('test dice %.4f, %.4f, %.4f, %.4f, %.4f, %.4f, %.4f' % tuple(testloss))
    print('best dice loss %.4f, %.4f, %.4f, %.4f, %.4f, %.4f, %.4f' % tuple(maxloss))
    with open(file_name_1, "a") as f:
        f.write(f"epoch {epoch+1}")
        f.write(f"\ttrain loss: {train_loss/(len(traindataloader)-j)}")
        f.write(f"\ttest loss: {test_loss/len(testdataloader)}\n")
                        
    
    with open(file_name_2, "a") as f:
        f.write(f"epoch {epoch+1}")
        f.write('\tDice coeff %.4f, %.4f, %.4f, %.4f, %.4f, %.4f, %.4f \n' % tuple(testloss))
    

    if (epoch+1)%10==0:
        
        t.save({
        'epoch': epoch,
        'Miccaimodel_state_dict':  SSABAM_Model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
        }, epoch150_path_name)
        
        with open(file_name, "a") as f:
            f.write(f"epoch {epoch+1}")
            f.write('\tDice coeff %.4f,%.4f,%.4f %.4f, %.4f, %.4f, %.4f \n' % tuple(testloss))
        



epoch  1


100%|███████████████████████████████████████████| 38/38 [01:38<00:00,  2.58s/it]


train loss:  3.876459518790637 



test loss 0.704241: 100%|███████████████████████| 10/10 [00:10<00:00,  1.01s/it]


test loss: 0.0
test dice 0.8395, 0.3492, 0.9213, 0.6059, 0.6110, 0.8647, 0.8683
best dice loss 0.8395, 0.3492, 0.9213, 0.6059, 0.6110, 0.8647, 0.8683
epoch  2


...

test loss: 0.0
test dice 0.8547, 0.5682, 0.9223, 0.7234, 0.7064, 0.8803, 0.8803
best dice loss 0.8620, 0.6195, 0.9396, 0.7283, 0.7225, 0.8847, 0.8849
epoch  50


100%|███████████████████████████████████████████| 38/38 [01:36<00:00,  2.54s/it]


train loss:  1.4884516739372262 



test loss 0.805980: 100%|███████████████████████| 10/10 [00:10<00:00,  1.01s/it]

test loss: 0.0
test dice 0.8597, 0.5724, 0.9297, 0.7214, 0.7165, 0.8802, 0.8818
best dice loss 0.8620, 0.6195, 0.9396, 0.7283, 0.7225, 0.8847, 0.8849


In [ ]:
# Evaluate
